In [ ]:
! sbatch_tail -p gpu -q gpu --gres=gpu:A100:4 --cpus-per-task 20 --mem-per-gpu 80G --ntasks-per-node=4 slurm/submit.sh "predict --config config/base_v3.yml --scprint_training.noise [0.7,1.0] --scprint_training.mask_ratio [] --model.dropout 0 --data.num_workers 17 --data.force_recompute_indices True --data.weight_scaler 0 --data.test_split 0 --data.batch_size 120 --trainer.num_nodes 1 --model.expr_emb_style metacell --data.get_knn_cells True --data.metacell_mode 0 --model.expr_encoder_layers 1 --model.compress_class_dim \"{'cell_type_ontology_term_id':64,'tissue_ontology_term_id':32,'disease_ontology_term_id':32,'age_group':8,'assay_ontology_term_id':12,'self_reported_ethnicity_ontology_term_id':8,'sex_ontology_term_id':2,'organism_ontology_term_id':8,'cell_culture':2}\" --model.layers_cls [256] --ckpt_path /pasteur/zeus/projets/p02/ml4ig_hot/Users/jkalfon/scprint_v2/czg34ita/checkpoints/last.ckpt"

In [ ]:
sbatch_tail -p gpu -q gpu --gres=gpu:1 --cpus-per-task 10 --mem-per-gpu 50G --ntasks-per-node=1 slurm/submit.sh "predict --config config/base_v3.yml --model.dropout 0 --data.num_workers 9 --data.force_recompute_indices True --data.weight_scaler 0 --data.test_split 0 --data.batch_size 60 --trainer.num_nodes 1 --model.expr_emb_style metacell --data.get_knn_cells True --data.metacell_mode 0 --model.expr_encoder_layers 1 --model.compress_class_dim \"{'cell_type_ontology_term_id':64,'tissue_ontology_term_id':32,'disease_ontology_term_id':32,'age_group':8,'assay_ontology_term_id':12,'self_reported_ethnicity_ontology_term_id':8,'sex_ontology_term_id':2,'organism_ontology_term_id':8,'cell_culture':2}\" --model.layers_cls [256] --ckpt_path /pasteur/zeus/projets/p02/ml4ig_hot/Users/jkalfon/scprint_v2/czg34ita/checkpoints/last.ckpt --data.store_location /pasteur/appa/scratch/jkalfon/predict_large"

## modified the model to replace logging of an anndata to logging of dataframe parquets 

In [ ]:
def log_adata(self, gtclass=None, name=""):
    """
    log_adata will log an adata from predictions.
    It will log to tensorboard and wandb if available

    see @utils.log_adata
    """
    try:
        mdir = self.logger.save_dir if self.logger.save_dir is not None else "/tmp"
    except:
        mdir = "data/"
    if not os.path.exists(mdir):
        os.makedirs(mdir)
    self.embs = torch.concat([self.embs[cl] for cl in self.classes], dim=1).cpu().numpy().astype(np.float16)
    self.pred = self.pred.cpu().numpy()
    loc = self.pred.argsort()[:, -3:][::-1]
    self.pred = self.pred[loc].astype(np.float16)
    loc = loc.astype(np.uint16)
    save = (str(mdir)
        + "/step_"
        + str(self.global_step)
        + "_"
        + str(self.name)
        + "_"
        + str(name)
        + "_"
        + str(self.global_rank)
    )
    loc.save(save+"_top3.npz")
    np.save(self.pred, save+"_pred.npz")
    np.save(self.embs, save+"_embs.npz")

In [ ]:
elems = [
    '',
    '',
]

In [ ]:
np.concatenate([np.load() for i in elems])

In [ ]:
embedder = ParametricUMAP(n_epochs = 200, verbose=True)

embedder.fit(train_images)

In [ ]:
fig, ax = plt.subplots()
ax.plot(embedder._history['loss'])
ax.set_ylabel('Cross Entropy')
ax.set_xlabel('Epoch')

In [ ]:
embedder.save('/tmp/model')


In [ ]:
from umap.parametric_umap import load_ParametricUMAP
embedder = load_ParametricUMAP('/tmp/model')

umap = embedder.transform(elems)
np.save(umap, '_umap.npz')